<a href="https://colab.research.google.com/github/GurionRamapoguSajeevan/credit-risk-modeling-ML-pipeline/blob/main/01_data_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>